In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup

In [2]:
# Extract "Article Name" from all "Article Name.html" for a list of article names
file_names = [name[:-5] for name in listdir('../wikiarticles30') if isfile(join('../wikiarticles30', name))]
file_names.remove(".DS_")
# turn list into df
wikiarticles = pd.DataFrame(file_names,columns=['article_name'])

In [3]:
def wiki_url_extractor(filename):
    '''
    Program to extract wgPageName from html of each site, then returns url
    Looking through html file to get index for wgPageName in html
    Then, iterating through the strings after to locate the next ", where the wgPageName ends
    Finally, returns the functional wikipedia url
    '''
    html_handle = open(filename)
    html_text = html_handle.read()
    wgPageName_index = html_text.find('"wgPageName":')

    # for loop to find the second quotation mark marking the end of wgPageName
    end_index = 0
    for i in range(wgPageName_index + 14, wgPageName_index + 500):
        if html_text[i] == '"':
            end_index= i
            break

    wgPageName_raw = html_text[wgPageName_index+14:end_index]
    wgPageName = wgPageName_raw.replace("'", "%27") #replace ' with %27
    
    url = "https://en.wikipedia.org/wiki/" + wgPageName
    return url

In [12]:
# add article url column for every article
for article_name in wikiarticles['article_name']:
    url_list = []
    url = wiki_url_extractor('../wikiarticles30/' + article_name + '.html')
    print(url)
    url_list.append(url)
    #wikiarticles['url'] = url_list
    

https://en.wikipedia.org/wiki/Slaughter_Trail
https://en.wikipedia.org/wiki/Sonny_Dee_Bar
https://en.wikipedia.org/wiki/Sproat_Lake
https://en.wikipedia.org/wiki/Sissi_–_The_Young_Empress
https://en.wikipedia.org/wiki/Spelungula
https://en.wikipedia.org/wiki/Slab_City,_Wisconsin
https://en.wikipedia.org/wiki/Sophia_Somajo
https://en.wikipedia.org/wiki/Single_visit_dentistry
https://en.wikipedia.org/wiki/Sredinny_Range
https://en.wikipedia.org/wiki/Spill_containment
https://en.wikipedia.org/wiki/South_Yorkshire_Fencing
https://en.wikipedia.org/wiki/Spawn_(novel)
https://en.wikipedia.org/wiki/Simpson%27s_Hospital,_Dublin
https://en.wikipedia.org/wiki/Socket_(video_game)
https://en.wikipedia.org/wiki/St_Clement%27s_Church,_Norwich
https://en.wikipedia.org/wiki/Southern_Defense_Command
https://en.wikipedia.org/wiki/Slovakia_men%27s_national_under-18_basketball_team
https://en.wikipedia.org/wiki/St_Bonaventure%27s_High_School
https://en.wikipedia.org/wiki/Sonargaon_Upazila
https://en.wikipe